# Libraries

In [7]:
import pandas as pd
import math
import numpy as np

# Generating dataset baseline - per state

In [ ]:
# gowalla/checkins.csv é o arquivo gowalla_geo_checkins.csv gerado pelo Paulo
g_dataset = pd.read_csv('checkins_data/checkins_local_Alabama.csv', index_col=False, usecols = ['userid', 'datetime', 'lat', 'lng', 'placeid', 'categoryid'])


In [9]:
g_dataset['country'] = 'United States'
g_dataset['state'] = 'Alabama'

In [10]:
# datetime converter
g_dataset['datetime'] = pd.to_datetime(g_dataset['datetime'])
order_cols = ['userid', 'categoryid', 'placeid', 'datetime', 'lat', 'lng', 'country', 'state']
g_dataset = g_dataset[order_cols]

In [11]:
cols = ['userid', 'category', 'placeid', 'local_datetime', 'latitude', 'longitude', 'country_name', 'state_name']
# Criar um dicionário que mapeia os nomes atuais das colunas para os novos nomes
maps = dict(zip(g_dataset.columns, cols))

# Renomear as colunas usando o método rename()
g_dataset = g_dataset.rename(columns=maps)

In [15]:
# category_names.csv é o arquivo category_names.csv gerado pelo Paulo

categories = pd.read_csv('checkins_data/category_name.csv')

In [16]:
g_dataset = pd.merge(g_dataset, categories, left_on="category", right_on="categoryid")
g_dataset['category'] = g_dataset['name']
g_dataset.drop(['name', 'categoryid'], axis=1, inplace=True)

É necessário remover os usuários que tenham visitado somente 1 local (independente do número de Checkins, o que importa é a quantidade de locais)

Este filtro é necessário, pois o Cláudio o aplica para gerar os dados de entrada

In [17]:
qt_lugares_por_usuario = (
    g_dataset
    .groupby('userid')
    .agg(
        {'placeid':'nunique'}
    )
    .sort_values(by=['placeid'], ascending=False)
    .reset_index()
)

# filtrando usuarios com menos de 30 checkins
qt_lugares_por_usuario = qt_lugares_por_usuario[qt_lugares_por_usuario['placeid'] >= 2]

usuarios_validos = qt_lugares_por_usuario['userid'].unique()

g_dataset = g_dataset[g_dataset['userid'].isin(usuarios_validos)]

In [9]:
g_dataset.to_csv('gowalla/checkins.csv', index=False)